In [1]:
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import openaq
import warnings

warnings.simplefilter('ignore')

%matplotlib inline

# Set major seaborn asthetics
sns.set("notebook", style='ticks', font_scale=1.0)

# Increase the quality of inline plots
mpl.rcParams['figure.dpi']= 500


pandas v1.1.0
matplotlib v3.3.0
seaborn v0.11.0
openaq v1.1.0


In [7]:
api = openaq.OpenAQ()
resp = api.cities(df=True, limit=10000)

# display the first 10 rows
resp.query("country == 'PL'")

,country,name,city,count,locations
2000,PL,Augustów,Augustów,87721,2
2001,PL,Belsk,Belsk,32105,1
2002,PL,Belsk Duży,Belsk Duży,86602,1
2003,PL,Biała,Biała,27538,1
2004,PL,Biała Podlaska,Biała Podlaska,106519,1
...,...,...,...,...,...
2170,PL,Zielonka,Zielonka,97764,1
2171,PL,Złoty Potok,Złoty Potok,132849,1
2172,PL,Żory,Żory,48221,1
2173,PL,Żyrardów,Żyrardów,45330,1


In [72]:
CITIES = ["Warszawa", "Kraków", "Poznań", "Katowice", "Białystok", "Elbląg"]

In [ ]:
LOCATIONS = ["Warszawa-Śródmieście", "Kraków-", "Poznań", "Katowice", "Białystok-Miejska", "WIOŚ Elbląg ul. Bażyńskiego"]

In [58]:
res = api.measurements(city="Warszawa", limit=100, df=True)
res.parameter.unique()

array(['so2', 'o3', 'co', 'no2', 'pm10', 'pm25', 'bc'], dtype=object)

In [66]:
PARAMETERS = ['pm25', 'pm10', 'no2', 'so2', 'o3', 'co']

In [88]:
import unicodedata

def strip_accents(text):
    return ''.join(c for c in unicodedata.normalize('NFKD', text.lower().replace("ł","l")) if unicodedata.category(c) != 'Mn')


In [120]:
res = api.locations(city="Warszawa", df=True)

res.iloc[0].location
res = api.measurements(city="Warszawa", location="Warszawa-Ursynów", df=True)

In [133]:
def check_params():
    return all(param in res.parameter.unique() for param in PARAMETERS) 

dfs = {}

for city in CITIES:
    # res = api.locations(city="Warszawa", df=True)

]
    res = api.measurements(city=city,limit=100, df=True)
    print(check_params())
    if check_params():

        dfs[strip_accents(city)] = res


True
True
True
True
True
True


In [176]:
dfs["poznan"].location.unique()

array(['Poznan-Dabrowskiego', 'Poznań - RatajeMOB', 'Poznan-Polanka'],
      dtype=object)

In [180]:
params = dfs["poznan"].loc[dfs["poznan"]["location"] == "Poznan-Polanka"]["parameter"].unique()
all(param in params for param in PARAMETERS)

False

In [204]:
dfs["bialystok"].loc[dfs["bialystok"]["parameter"] == "o3"].mean()

value                    28.222222
coordinates.latitude     53.129306
coordinates.longitude    23.181744
dtype: float64

In [213]:
dfs["bialystok"].index=pd.to_datetime(dfs["bialystok"].index)


In [224]:
dfs["bialystok"]

,location,parameter,value,unit,country,city,date.utc,coordinates.latitude,coordinates.longitude
date.local,,,,,,,,,
2020-12-21 23:00:00,Białystok-Miejska,no2,8.60,b'\xc2\xb5g/m\xc2\xb3',PL,Białystok,2020-12-21 22:00:00+00:00,53.126689,23.155869
2020-12-21 23:00:00,Białystok-Warszawska,o3,21.00,b'\xc2\xb5g/m\xc2\xb3',PL,Białystok,2020-12-21 22:00:00+00:00,53.129306,23.181744
2020-12-21 23:00:00,Białystok-Warszawska,pm10,37.35,b'\xc2\xb5g/m\xc2\xb3',PL,Białystok,2020-12-21 22:00:00+00:00,53.129306,23.181744
2020-12-21 23:00:00,Białystok-Miejska,bc,0.94,b'\xc2\xb5g/m\xc2\xb3',PL,Białystok,2020-12-21 22:00:00+00:00,53.126689,23.155869
2020-12-21 23:00:00,Białystok-Miejska,co,396.00,b'\xc2\xb5g/m\xc2\xb3',PL,Białystok,2020-12-21 22:00:00+00:00,53.126689,23.155869
...,...,...,...,...,...,...,...,...,...
2020-12-21 08:00:00,Białystok-Warszawska,o3,31.00,b'\xc2\xb5g/m\xc2\xb3',PL,Białystok,2020-12-21 07:00:00+00:00,53.129306,23.181744
2020-12-21 07:00:00,Białystok-Warszawska,o3,36.00,b'\xc2\xb5g/m\xc2\xb3',PL,Białystok,2020-12-21 06:00:00+00:00,53.129306,23.181744
2020-12-21 07:00:00,Białystok-Warszawska,pm10,18.45,b'\xc2\xb5g/m\xc2\xb3',PL,Białystok,2020-12-21 06:00:00+00:00,53.129306,23.181744


In [223]:
df_avg = dfs["katowice"].resample('D').mean()
df_avg.pop("value")


date.local
2020-12-21    144.873923
Freq: D, Name: value, dtype: float64